## 3. ORM, RE

#### SQLAlchemy ORM

In [1]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey

engine = create_engine('sqlite:///ex4.db', echo = True)

base = declarative_base()

In [3]:
class User(base):
    __tablename__ = 'User'
    
    uid = Column('uid', Integer, primary_key = True)
    name = Column('name', String, nullable = False)
    password = Column('pass', String, nullable = False)
    
    def __repr__(self):
        return 'UID={0}, NAME = {1}, PASS = {2}'.format(
        self.uid, self.name, self.password)

In [4]:
# in-memory Table 객체들 -> 실제 database Table로 생성
base.metadata.create_all(bind=engine)

2020-07-15 10:38:44,176 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-15 10:38:44,184 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 10:38:44,186 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-15 10:38:44,187 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 10:38:44,188 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("User")
2020-07-15 10:38:44,189 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 10:38:44,192 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "User" (
	uid INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	pass VARCHAR NOT NULL, 
	PRIMARY KEY (uid)
)


2020-07-15 10:38:44,193 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 10:38:44,203 INFO sqlalchemy.engine.base.Engine COMMIT


In [5]:
# # Metadata Class에 등록된(생성한) table 객체 초기화할 때
# base.metadata.clear()

In [6]:
base.metadata.tables

immutabledict({'User': Table('User', MetaData(bind=None), Column('uid', Integer(), table=<User>, primary_key=True, nullable=False), Column('name', String(), table=<User>, nullable=False), Column('pass', String(), table=<User>, nullable=False), schema=None)})

In [7]:
user2 = User(name = 'test1', password = 'testtest1')

user2

UID=None, NAME = test1, PASS = testtest1

In [2]:
from sqlalchemy.orm import sessionmaker # session을 통한 통신

Session = sessionmaker(bind = engine)
session = Session()

In [ ]:
session.add(user2)

In [9]:
session.commit()

2020-07-15 10:38:47,865 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 10:38:47,866 INFO sqlalchemy.engine.base.Engine INSERT INTO "User" (name, pass) VALUES (?, ?)
2020-07-15 10:38:47,867 INFO sqlalchemy.engine.base.Engine ('test1', 'testtest1')
2020-07-15 10:38:47,870 INFO sqlalchemy.engine.base.Engine COMMIT


In [10]:
session.dirty, session.is_modified

(IdentitySet([]),
 <bound method Session.is_modified of <sqlalchemy.orm.session.Session object at 0x000001451D7A4B38>>)

In [11]:
user2.name = 'changename'

In [12]:
session.dirty, session.is_modified

2020-07-15 10:38:50,146 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 10:38:50,147 INFO sqlalchemy.engine.base.Engine SELECT "User".pass AS "User_pass", "User".uid AS "User_uid" 
FROM "User" 
WHERE "User".uid = ?
2020-07-15 10:38:50,148 INFO sqlalchemy.engine.base.Engine (1,)


(IdentitySet([UID=1, NAME = changename, PASS = testtest1]),
 <bound method Session.is_modified of <sqlalchemy.orm.session.Session object at 0x000001451D7A4B38>>)

In [13]:
session.commit()

2020-07-15 10:38:50,528 INFO sqlalchemy.engine.base.Engine UPDATE "User" SET name=? WHERE "User".uid = ?
2020-07-15 10:38:50,529 INFO sqlalchemy.engine.base.Engine ('changename', 1)
2020-07-15 10:38:50,534 INFO sqlalchemy.engine.base.Engine COMMIT


In [14]:
user2

2020-07-15 10:38:50,945 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 10:38:50,947 INFO sqlalchemy.engine.base.Engine SELECT "User".pass AS "User_pass", "User".uid AS "User_uid", "User".name AS "User_name" 
FROM "User" 
WHERE "User".uid = ?
2020-07-15 10:38:50,947 INFO sqlalchemy.engine.base.Engine (1,)


UID=1, NAME = changename, PASS = testtest1

In [15]:
session.add_all([User(name='test2', password='asdfas'),
                User(name='test3', password='asfjdd')])

In [16]:
list(session.query(User))

2020-07-15 10:38:52,726 INFO sqlalchemy.engine.base.Engine INSERT INTO "User" (name, pass) VALUES (?, ?)
2020-07-15 10:38:52,727 INFO sqlalchemy.engine.base.Engine ('test2', 'asdfas')
2020-07-15 10:38:52,731 INFO sqlalchemy.engine.base.Engine INSERT INTO "User" (name, pass) VALUES (?, ?)
2020-07-15 10:38:52,731 INFO sqlalchemy.engine.base.Engine ('test3', 'asfjdd')
2020-07-15 10:38:52,732 INFO sqlalchemy.engine.base.Engine SELECT "User".pass AS "User_pass", "User".uid AS "User_uid", "User".name AS "User_name" 
FROM "User"
2020-07-15 10:38:52,733 INFO sqlalchemy.engine.base.Engine ()


[UID=1, NAME = changename, PASS = testtest1,
 UID=2, NAME = test2, PASS = asdfas,
 UID=3, NAME = test3, PASS = asfjdd]

In [17]:
list(session.query(User).filter(User.name=='test2'))

2020-07-15 10:38:53,502 INFO sqlalchemy.engine.base.Engine SELECT "User".pass AS "User_pass", "User".uid AS "User_uid", "User".name AS "User_name" 
FROM "User" 
WHERE "User".name = ?
2020-07-15 10:38:53,503 INFO sqlalchemy.engine.base.Engine ('test2',)


[UID=2, NAME = test2, PASS = asdfas]

In [18]:
list(session.query(User).filter(User.name.like('test%')))

2020-07-15 10:38:54,488 INFO sqlalchemy.engine.base.Engine SELECT "User".pass AS "User_pass", "User".uid AS "User_uid", "User".name AS "User_name" 
FROM "User" 
WHERE "User".name LIKE ?
2020-07-15 10:38:54,489 INFO sqlalchemy.engine.base.Engine ('test%',)


[UID=2, NAME = test2, PASS = asdfas, UID=3, NAME = test3, PASS = asfjdd]

In [3]:
base.metadata.clear()

In [4]:
base.metadata.tables

immutabledict({})

In [5]:
class Artist(base):
    __tablename__ = 'ARTIST'
    
    pk = Column('ID', Integer, primary_key = True)
    name = Column('NAME', String)
    
    def __repr__(self):
        return 'PK:{0}, NAME : {1}'.format(self.pk, self.name)

class Genre(base):
    __tablename__ = 'GENRE'
    
    pk = Column('ID', Integer, primary_key = True)
    name = Column('NAME', String)
    
    def __repr__(self):
        return 'PK:{0}, NAME : {1}'.format(self.pk, self.name)

class Album(base):
    __tablename__ = 'ALBUM'
    
    pk = Column('ID', Integer, primary_key = True)
    name = Column('NAME', String)
    fk = Column('FK', Integer, ForeignKey('ARTIST.ID'))
    
    def __repr__(self):
        return 'PK:{0}, NAME : {1}, FK : {2}'.format(self.pk, self.name, self.fk)

class Track(base):
    __tablename__ = 'TRACK'
    
    pk = Column('ID', Integer, primary_key = True)
    name = Column('NAME', String)
    fk1 = Column('FK1', Integer, ForeignKey('ALBUM.ID'))
    fk2 = Column('FK2', Integer, ForeignKey('GENRE.ID'))
    
    def __repr__(self):
        return 'PK:{0}, NAME : {1}, FK1 : {2}, FK2 : {3}'.format(self.pk, self.name, self.fk1, self.fk2)

In [6]:
base.metadata.tables

immutabledict({'ARTIST': Table('ARTIST', MetaData(bind=None), Column('ID', Integer(), table=<ARTIST>, primary_key=True, nullable=False), Column('NAME', String(), table=<ARTIST>), schema=None), 'GENRE': Table('GENRE', MetaData(bind=None), Column('ID', Integer(), table=<GENRE>, primary_key=True, nullable=False), Column('NAME', String(), table=<GENRE>), schema=None), 'ALBUM': Table('ALBUM', MetaData(bind=None), Column('ID', Integer(), table=<ALBUM>, primary_key=True, nullable=False), Column('NAME', String(), table=<ALBUM>), Column('FK', Integer(), ForeignKey('ARTIST.ID'), table=<ALBUM>), schema=None), 'TRACK': Table('TRACK', MetaData(bind=None), Column('ID', Integer(), table=<TRACK>, primary_key=True, nullable=False), Column('NAME', String(), table=<TRACK>), Column('FK1', Integer(), ForeignKey('ALBUM.ID'), table=<TRACK>), Column('FK2', Integer(), ForeignKey('GENRE.ID'), table=<TRACK>), schema=None)})

In [7]:
# in-memory Table 객체들 -> 실제 database Table로 생성
base.metadata.create_all(bind=engine)

2020-07-15 11:09:12,951 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-15 11:09:12,958 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:09:12,959 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-15 11:09:12,960 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:09:12,961 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ARTIST")
2020-07-15 11:09:12,961 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:09:12,963 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("GENRE")
2020-07-15 11:09:12,964 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:09:12,964 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ALBUM")
2020-07-15 11:09:12,965 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:09:12,966 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("TRACK")
2020-07-15 11:09:12,966 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:09:12,968 INFO sqlalchemy.engin

In [8]:
a1 = Artist(name = '1')
a2 = Artist(name = '2')
session.add_all([a1, a2])
session.commit()

2020-07-15 11:09:13,354 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 11:09:13,356 INFO sqlalchemy.engine.base.Engine INSERT INTO "ARTIST" ("NAME") VALUES (?)
2020-07-15 11:09:13,356 INFO sqlalchemy.engine.base.Engine ('1',)
2020-07-15 11:09:13,360 INFO sqlalchemy.engine.base.Engine INSERT INTO "ARTIST" ("NAME") VALUES (?)
2020-07-15 11:09:13,361 INFO sqlalchemy.engine.base.Engine ('2',)
2020-07-15 11:09:13,362 INFO sqlalchemy.engine.base.Engine COMMIT


In [9]:
g1 = Genre(name = 'g1')
g2 = Genre(name = 'g2')
session.add_all([g1,g2])
session.commit()

2020-07-15 11:09:13,875 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 11:09:13,877 INFO sqlalchemy.engine.base.Engine INSERT INTO "GENRE" ("NAME") VALUES (?)
2020-07-15 11:09:13,877 INFO sqlalchemy.engine.base.Engine ('g1',)
2020-07-15 11:09:13,880 INFO sqlalchemy.engine.base.Engine INSERT INTO "GENRE" ("NAME") VALUES (?)
2020-07-15 11:09:13,880 INFO sqlalchemy.engine.base.Engine ('g2',)
2020-07-15 11:09:13,881 INFO sqlalchemy.engine.base.Engine COMMIT


In [10]:
session.add_all([
    Album(name = 'album1', fk = a1.pk),
    Album(name = 'album2', fk = a2.pk)
])
session.commit()

2020-07-15 11:09:14,296 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 11:09:14,299 INFO sqlalchemy.engine.base.Engine SELECT "ARTIST"."ID" AS "ARTIST_ID", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."ID" = ?
2020-07-15 11:09:14,300 INFO sqlalchemy.engine.base.Engine (1,)
2020-07-15 11:09:14,302 INFO sqlalchemy.engine.base.Engine SELECT "ARTIST"."ID" AS "ARTIST_ID", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."ID" = ?
2020-07-15 11:09:14,303 INFO sqlalchemy.engine.base.Engine (2,)
2020-07-15 11:09:14,305 INFO sqlalchemy.engine.base.Engine INSERT INTO "ALBUM" ("NAME", "FK") VALUES (?, ?)
2020-07-15 11:09:14,305 INFO sqlalchemy.engine.base.Engine ('album1', 1)
2020-07-15 11:09:14,307 INFO sqlalchemy.engine.base.Engine INSERT INTO "ALBUM" ("NAME", "FK") VALUES (?, ?)
2020-07-15 11:09:14,308 INFO sqlalchemy.engine.base.Engine ('album2', 2)
2020-07-15 11:09:14,309 INFO sqlalchemy.engine.base.Engine COMMIT


In [11]:
a11 = session.query(Album).filter(Album.fk == a1.pk).one()
a12 = session.query(Album).filter(Album.fk == a2.pk).one()

2020-07-15 11:09:14,624 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 11:09:14,625 INFO sqlalchemy.engine.base.Engine SELECT "ARTIST"."ID" AS "ARTIST_ID", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."ID" = ?
2020-07-15 11:09:14,626 INFO sqlalchemy.engine.base.Engine (1,)
2020-07-15 11:09:14,628 INFO sqlalchemy.engine.base.Engine SELECT "ALBUM"."ID" AS "ALBUM_ID", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE "ALBUM"."FK" = ?
2020-07-15 11:09:14,629 INFO sqlalchemy.engine.base.Engine (1,)
2020-07-15 11:09:14,631 INFO sqlalchemy.engine.base.Engine SELECT "ARTIST"."ID" AS "ARTIST_ID", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."ID" = ?
2020-07-15 11:09:14,631 INFO sqlalchemy.engine.base.Engine (2,)
2020-07-15 11:09:14,633 INFO sqlalchemy.engine.base.Engine SELECT "ALBUM"."ID" AS "ALBUM_ID", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE "ALBUM"."FK" = ?
2020-07-15 1

In [12]:
session.add_all([
    Track(name='track1', fk1=a11.pk, fk2=g1.pk),
    Track(name='track2', fk1=a12.pk, fk2=g2.pk),
    Track(name='track3', fk1=a11.pk, fk2=g1.pk),
    Track(name='track4', fk1=a12.pk, fk2=g2.pk)
])
session.commit()

2020-07-15 11:09:14,937 INFO sqlalchemy.engine.base.Engine SELECT "GENRE"."ID" AS "GENRE_ID", "GENRE"."NAME" AS "GENRE_NAME" 
FROM "GENRE" 
WHERE "GENRE"."ID" = ?
2020-07-15 11:09:14,938 INFO sqlalchemy.engine.base.Engine (1,)
2020-07-15 11:09:14,940 INFO sqlalchemy.engine.base.Engine SELECT "GENRE"."ID" AS "GENRE_ID", "GENRE"."NAME" AS "GENRE_NAME" 
FROM "GENRE" 
WHERE "GENRE"."ID" = ?
2020-07-15 11:09:14,940 INFO sqlalchemy.engine.base.Engine (2,)
2020-07-15 11:09:14,942 INFO sqlalchemy.engine.base.Engine INSERT INTO "TRACK" ("NAME", "FK1", "FK2") VALUES (?, ?, ?)
2020-07-15 11:09:14,943 INFO sqlalchemy.engine.base.Engine ('track1', 1, 1)
2020-07-15 11:09:14,945 INFO sqlalchemy.engine.base.Engine INSERT INTO "TRACK" ("NAME", "FK1", "FK2") VALUES (?, ?, ?)
2020-07-15 11:09:14,946 INFO sqlalchemy.engine.base.Engine ('track2', 2, 2)
2020-07-15 11:09:14,947 INFO sqlalchemy.engine.base.Engine INSERT INTO "TRACK" ("NAME", "FK1", "FK2") VALUES (?, ?, ?)
2020-07-15 11:09:14,948 INFO sqlalche

In [13]:
list(session.query(Track))

2020-07-15 11:09:15,276 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 11:09:15,277 INFO sqlalchemy.engine.base.Engine SELECT "TRACK"."ID" AS "TRACK_ID", "TRACK"."NAME" AS "TRACK_NAME", "TRACK"."FK1" AS "TRACK_FK1", "TRACK"."FK2" AS "TRACK_FK2" 
FROM "TRACK"
2020-07-15 11:09:15,277 INFO sqlalchemy.engine.base.Engine ()


[PK:1, NAME : track1, FK1 : 1, FK2 : 1,
 PK:2, NAME : track2, FK1 : 2, FK2 : 2,
 PK:3, NAME : track3, FK1 : 1, FK2 : 1,
 PK:4, NAME : track4, FK1 : 2, FK2 : 2]

In [14]:
tracklist = session.query(Track).all()

2020-07-15 11:09:15,623 INFO sqlalchemy.engine.base.Engine SELECT "TRACK"."ID" AS "TRACK_ID", "TRACK"."NAME" AS "TRACK_NAME", "TRACK"."FK1" AS "TRACK_FK1", "TRACK"."FK2" AS "TRACK_FK2" 
FROM "TRACK"
2020-07-15 11:09:15,624 INFO sqlalchemy.engine.base.Engine ()


In [15]:
a1, a2, a11, a12, g1, g2, tracklist

2020-07-15 11:09:16,158 INFO sqlalchemy.engine.base.Engine SELECT "ARTIST"."ID" AS "ARTIST_ID", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."ID" = ?
2020-07-15 11:09:16,158 INFO sqlalchemy.engine.base.Engine (1,)
2020-07-15 11:09:16,160 INFO sqlalchemy.engine.base.Engine SELECT "ARTIST"."ID" AS "ARTIST_ID", "ARTIST"."NAME" AS "ARTIST_NAME" 
FROM "ARTIST" 
WHERE "ARTIST"."ID" = ?
2020-07-15 11:09:16,161 INFO sqlalchemy.engine.base.Engine (2,)
2020-07-15 11:09:16,163 INFO sqlalchemy.engine.base.Engine SELECT "ALBUM"."ID" AS "ALBUM_ID", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE "ALBUM"."ID" = ?
2020-07-15 11:09:16,164 INFO sqlalchemy.engine.base.Engine (1,)
2020-07-15 11:09:16,166 INFO sqlalchemy.engine.base.Engine SELECT "ALBUM"."ID" AS "ALBUM_ID", "ALBUM"."NAME" AS "ALBUM_NAME", "ALBUM"."FK" AS "ALBUM_FK" 
FROM "ALBUM" 
WHERE "ALBUM"."ID" = ?
2020-07-15 11:09:16,167 INFO sqlalchemy.engine.base.Engine (2,)
2020-07-15 11:09:16,169 

(PK:1, NAME : 1,
 PK:2, NAME : 2,
 PK:1, NAME : album1, FK : 1,
 PK:2, NAME : album2, FK : 2,
 PK:1, NAME : g1,
 PK:2, NAME : g2,
 [PK:1, NAME : track1, FK1 : 1, FK2 : 1,
  PK:2, NAME : track2, FK1 : 2, FK2 : 2,
  PK:3, NAME : track3, FK1 : 1, FK2 : 1,
  PK:4, NAME : track4, FK1 : 2, FK2 : 2])

In [48]:
result = session.query(Artist.name, Album.name, Genre.name, Track.name)\
.select_from(Track)\
.join(Genre).join(Album).join(Artist).all()
for _ in result:
    print(['Artist:{0}, Album:{1}, Genre:{2}, Track:{3}'.format(
    _[0],_[1],_[2],_[3])])

2020-07-15 11:31:58,671 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 11:31:58,673 INFO sqlalchemy.engine.base.Engine SELECT "ARTIST"."NAME" AS "ARTIST_NAME", "ALBUM"."NAME" AS "ALBUM_NAME", "GENRE"."NAME" AS "GENRE_NAME", "TRACK"."NAME" AS "TRACK_NAME" 
FROM "TRACK" JOIN "GENRE" ON "GENRE"."ID" = "TRACK"."FK2" JOIN "ALBUM" ON "ALBUM"."ID" = "TRACK"."FK1" JOIN "ARTIST" ON "ARTIST"."ID" = "ALBUM"."FK"
2020-07-15 11:31:58,675 INFO sqlalchemy.engine.base.Engine ()
['Artist:1, Album:album1, Genre:g1, Track:track1']
['Artist:2, Album:album2, Genre:g2, Track:track2']
['Artist:1, Album:album1, Genre:g1, Track:track3']
['Artist:2, Album:album2, Genre:g2, Track:track4']


In [90]:
base.metadata.tables

immutabledict({'Artist': Table('Artist', MetaData(bind=Engine(sqlite:///ex4.db)), Column('ID', Integer(), table=<Artist>, primary_key=True, nullable=False), Column('NAME', String(), table=<Artist>), schema=None), 'Genre': Table('Genre', MetaData(bind=Engine(sqlite:///ex4.db)), Column('ID', Integer(), table=<Genre>, primary_key=True, nullable=False), Column('NAME', String(), table=<Genre>), schema=None), 'ALBUM': Table('ALBUM', MetaData(bind=Engine(sqlite:///ex4.db)), Column('ID', Integer(), table=<ALBUM>, primary_key=True, nullable=False), Column('NAME', String(), table=<ALBUM>), Column('FK', Integer(), ForeignKey('ARTIST.ID'), table=<ALBUM>), schema=None), 'Track': Table('Track', MetaData(bind=Engine(sqlite:///ex4.db)), Column('ID', Integer(), table=<Track>, primary_key=True, nullable=False), Column('NAME', String(), table=<Track>), Column('FK1', Integer(), ForeignKey('ALBUM.ID'), table=<Track>), Column('FK2', Integer(), ForeignKey('GENRE.ID'), table=<Track>), schema=None)})

In [14]:
base.metadata.clear()

### Relationship

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

In [2]:
engine = create_engine('sqlite:///ex4.db', echo = True)
base = declarative_base(bind = engine)

In [3]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
class Artist(base):
    __tablename__ = 'Artist'
    
    pk = Column('ID', Integer, primary_key = True)
    name = Column('NAME', String)
    albumlist = relationship('Album', back_populates='artist')
    
    def __repr__(self):
        return 'PK:{0}, NAME : {1}'.format(self.pk, self.name)

class Genre(base):
    __tablename__ = 'Genre'
    
    pk = Column('ID', Integer, primary_key = True)
    name = Column('NAME', String)
    tracklist = relationship('Track', back_populates='genre')
    
    def __repr__(self):
        return 'PK:{0}, NAME : {1}'.format(self.pk, self.name)

class Album(base):
    __tablename__ = 'Album'
    
    pk = Column('ID', Integer, primary_key = True)
    name = Column('NAME', String)
    fk = Column('FK', Integer, ForeignKey('Artist.ID'))
    artist = relationship('Artist', back_populates='albumlist', uselist = False)
    tracklist = relationship('Track', back_populates='album')
    
    def __repr__(self):
        return 'PK:{0}, NAME : {1}, FK : {2}'.format(self.pk, self.name, self.fk)

class Track(base):
    __tablename__ = 'Track'
    
    pk = Column('ID', Integer, primary_key = True)
    name = Column('NAME', String)
    fk1 = Column('FK1', Integer, ForeignKey('Album.ID'))
    fk2 = Column('FK2', Integer, ForeignKey('Genre.ID'))
    album = relationship('Album', back_populates='tracklist', uselist=False)
    genre = relationship('Genre', back_populates='tracklist', uselist=False)
    
    
    def __repr__(self):
        return 'PK:{0}, NAME : {1}, FK1 : {2}, FK2 : {3}'.format(self.pk, self.name, self.fk1, self.fk2)

In [5]:
session.query(Artist).one()

2020-07-15 11:52:44,544 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-15 11:52:44,551 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:52:44,552 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-15 11:52:44,553 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:52:44,554 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-07-15 11:52:44,555 INFO sqlalchemy.engine.base.Engine SELECT "Artist"."ID" AS "Artist_ID", "Artist"."NAME" AS "Artist_NAME" 
FROM "Artist"
2020-07-15 11:52:44,556 INFO sqlalchemy.engine.base.Engine ()


OperationalError: (sqlite3.OperationalError) no such table: Artist
[SQL: SELECT "Artist"."ID" AS "Artist_ID", "Artist"."NAME" AS "Artist_NAME" 
FROM "Artist"]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [62]:
base.metadata.create_all(bind = engine)

2020-07-15 11:35:01,357 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-15 11:35:01,358 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:35:01,359 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-15 11:35:01,360 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:35:01,362 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ARTIST")
2020-07-15 11:35:01,362 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:35:01,364 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("GENRE")
2020-07-15 11:35:01,365 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:35:01,367 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("ALBUM")
2020-07-15 11:35:01,368 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:35:01,369 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("TRACK")
2020-07-15 11:35:01,370 INFO sqlalchemy.engine.base.Engine ()


In [6]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, backref

In [7]:
engine = create_engine('sqlite:///playlist.db', echo = True)
base = declarative_base(bind = engine)

In [8]:
base.metadata.clear()

In [9]:
class Artist(base):
    __tablename__ = 'Artist'
    
    pk = Column('id', Integer, primary_key = True)
    name = Column('title', String)
    albumlist = relationship('Album', back_populates='artist')
    
    def __repr__(self):
        return 'PK:{0}, NAME : {1}'.format(self.pk, self.name)

class Genre(base):
    __tablename__ = 'Genre'
    
    pk = Column('id', Integer, primary_key = True)
    name = Column('title', String)
    tracklist = relationship('Track', back_populates='genre')
    
    def __repr__(self):
        return 'PK:{0}, NAME : {1}'.format(self.pk, self.name)

class Album(base):
    __tablename__ = 'Album'
    
    pk = Column('id', Integer, primary_key = True)
    name = Column('title', String)
    fk = Column('Artist_id', Integer, ForeignKey('Artist.id'))
    artist = relationship('Artist', back_populates='albumlist', uselist = False)
    tracklist = relationship('Track', back_populates='album')
    
    def __repr__(self):
        return 'PK:{0}, NAME : {1}, FK : {2}'.format(self.pk, self.name, self.fk)

class Track(base):
    __tablename__ = 'Track'
    
    pk = Column('id', Integer, primary_key = True)
    name = Column('title', String)
    fk1 = Column('Album_id', Integer, ForeignKey('Album.id'))
    fk2 = Column('Genre_id', Integer, ForeignKey('Genre.id'))
    album = relationship('Album', back_populates='tracklist', uselist=False)
    genre = relationship('Genre', back_populates='tracklist', uselist=False)
    
    
    def __repr__(self):
        return 'PK:{0}, NAME : {1}, FK1 : {2}, FK2 : {3}'.format(self.pk, self.name, self.fk1, self.fk2)

In [10]:
base.metadata.create_all(bind = engine)

2020-07-15 11:57:16,826 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-07-15 11:57:16,827 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:57:16,828 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-07-15 11:57:16,829 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:57:16,830 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2020-07-15 11:57:16,830 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:57:16,857 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2020-07-15 11:57:16,858 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:57:16,859 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2020-07-15 11:57:16,859 INFO sqlalchemy.engine.base.Engine ()
2020-07-15 11:57:16,861 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2020-07-15 11:57:16,861 INFO sqlalchemy.engine.base.Engine ()


In [11]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [13]:
genrelist = session.query(Genre).all()

2020-07-15 11:57:49,075 INFO sqlalchemy.engine.base.Engine SELECT "Genre".id AS "Genre_id", "Genre".title AS "Genre_title" 
FROM "Genre"
2020-07-15 11:57:49,077 INFO sqlalchemy.engine.base.Engine ()


In [16]:
genrelist[1].tracklist[0], 

(PK:3, NAME : About to Rock, FK1 : 1, FK2 : 2,
 [PK:3, NAME : About to Rock, FK1 : 1, FK2 : 2,
  PK:4, NAME : Who Made Who, FK1 : 1, FK2 : 2])

In [20]:
genrelist

[PK:1, NAME : Rock, PK:2, NAME : Metal]

In [29]:
genrelist[1].tracklist

[PK:3, NAME : About to Rock, FK1 : 1, FK2 : 2,
 PK:4, NAME : Who Made Who, FK1 : 1, FK2 : 2]

## HTML

## JSON, XML